## Data parsing script

This script aims at parsing data from various online macroeconomic databases.

In [92]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import pandasdmx as sdmx

### OECD

See Guide: https://data.oecd.org/api/

The API uses the standard SDMX REST API for queries. To create an SDMX-JSON query, a dataset identifier, a list of dimension item identifiers and some additional parameters must be supplied in an URL in the following format:[]
    
Using their identifiers, the list of desired dimension values to be included in the result. To get all available dimension values, use the “all” keyword. Dimensions should be separated by a dot (“.”) and for each dimension its values should be separated by the plus sign (“+”). If for a particular dimension, no dimension value identifiers are specified, then all available values of this dimension will be returned. To get the list of dimensions and dimension values, use:
http://stats.oecd.org/restsdmx/sdmx.ashx/GetDataStructure/<dataset identifier>

The request specifies four dimensions:1)Location, 2)Subject, 3)Measure, 4)Frequency

In [85]:
OECD_ROOT_URL = 'https://stats.oecd.org/sdmx-json/data'
def get_OECD_data(dataset,dimensions, params = None, root_dir=OECD_ROOT_URL):
    if not params:
        params = {}
    dim_args = ['+'.join(d) for d in dimensions]
    dim_str = '.'.join(dim_args)
    url = root_dir + '/' + dataset + '/' + dim_str + '/all'
    print('Requesting URL: {}'.format(url))
    return requests.get(url,params = params)

response = get_OECD_data('QNA',((),('PPPGDP','GDP','B1_GE'),('CUR','VOBARSA'),('Q')))
response 
#response.json()

Requesting URL: https://stats.oecd.org/sdmx-json/data/QNA/.PPPGDP+GDP+B1_GE.CUR+VOBARSA.Q/all


<Response [200]>

In [86]:
response.json()['dataSets'][0]['series'].keys()

dict_keys(['0:0:0:0', '1:0:0:0', '2:0:0:0', '3:0:0:0', '4:0:0:0', '5:0:0:0', '6:0:0:0', '7:0:0:0', '8:0:0:0', '9:0:0:0', '10:0:0:0', '11:0:0:0', '12:0:0:0', '13:0:0:0', '14:0:0:0', '15:0:0:0', '16:0:0:0', '17:0:0:0', '18:0:0:0', '19:0:0:0', '20:0:0:0', '21:0:0:0', '22:0:0:0', '23:0:0:0', '24:0:0:0', '25:0:0:0', '26:0:0:0', '27:0:0:0', '28:0:0:0', '29:0:0:0', '30:0:0:0', '31:0:0:0', '32:0:0:0', '33:0:0:0', '34:0:0:0', '35:0:0:0', '36:0:0:0', '37:0:0:0', '38:0:0:0', '39:0:0:0', '40:0:0:0', '41:0:0:0', '42:0:0:0', '43:0:0:0', '44:0:0:0', '45:0:0:0', '46:0:0:0', '47:0:0:0', '48:0:0:0'])

In [89]:
pd.DataFrame(response.json()['dataSets'][0]['series']['0:0:0:0']['observations']).T.head()

,0,1
0,233180.867087,NaN
1,239643.494379,NaN
2,240180.819883,NaN
3,239698.699054,NaN
4,240423.720453,NaN


with pandaSDMX:

In [106]:
oecd = sdmx.Request('OECD')
data_response = oecd.data(resource_id='QNA', key='all?startTime=2018')
df = data_response.write(data_response.data.series, parse_time=False)
df

LOCATION          JPN                                                 \
SUBJECT         GFSPB                                                  
MEASURE         CARSA                     CQR                  CQRSA   
FREQUENCY           A           Q           A          Q           A   
2018       27063000.0         NaN  27063000.0        NaN  27063000.0   
2018-Q1           NaN  27497500.0         NaN  7655200.0         NaN   
2018-Q2           NaN  27302800.0         NaN  5630800.0         NaN   
2018-Q3           NaN  26867700.0         NaN  6390900.0         NaN   
2018-Q4           NaN  26670500.0         NaN  7386100.0         NaN   
2019-Q1           NaN  27115800.0         NaN  7542800.0         NaN   
2019-Q2           NaN  27730400.0         NaN  5756300.0         NaN   

LOCATION                                                             ...  \
SUBJECT                                                              ...   
MEASURE                  LNBARSA                   LNBQR             ...   
FREQUENCY          Q           A           Q           A          Q  ...   
2018             NaN  25155700.0         NaN  25155700.0        NaN  ...   
2018-Q1    6874375.0         NaN  25741800.0         NaN  7186200.0  ...   
2018-Q2    6825700.0         NaN  25383300.0         NaN  5243300.0  ...   
2018-Q3    6716925.0         NaN  24924400.0         NaN  5939100.0  ...   
2018-Q4    6667625.0         NaN  24637600.0         NaN  6787100.0  ...   
2019-Q1    6778950.0         NaN  25006400.0         NaN  6973500.0  ...   
2019-Q2    6932600.0         NaN  25446800.0         NaN  5292000.0  ...   

LOCATION        AUS                                                            \
SUBJECT        P51A               P52                                           
MEASURE     LNBQRSA               CQR           CQRSA           LNBQR           
FREQUENCY         A         Q       A       Q       A       Q       A       Q   
2018       445047.0       NaN  2729.0     NaN  2729.0     NaN  2393.0     NaN   
2018-Q1         NaN  111355.0     NaN -2441.0     NaN   142.0     NaN -2402.0   
2018-Q2         NaN  111761.0     NaN  -987.0     NaN  1416.0     NaN  -865.0   
2018-Q3         NaN  111747.0     NaN  1956.0     NaN   171.0     NaN  2116.0   
2018-Q4         NaN  110303.0     NaN  4201.0     NaN   920.0     NaN  3544.0   
2019-Q1         NaN  109183.0     NaN -2026.0     NaN   696.0     NaN -1754.0   
2019-Q2         NaN  107275.0     NaN -4708.0     NaN -2411.0     NaN     NaN   

LOCATION                   
SUBJECT                    
MEASURE   LNBQRSA          
FREQUENCY       A       Q  
2018       2393.0     NaN  
2018-Q1       NaN   578.0  
2018-Q2       NaN  1375.0  
2018-Q3       NaN    18.0  
2018-Q4       NaN   846.0  
2019-Q1       NaN   559.0  
2019-Q2       NaN -1982.0  

[7 rows x 41120 columns]

In [104]:
df.melt().SUBJECT.unique()

array(['GFSPB', 'P6', 'P61', 'P62', 'P7', 'P71', 'P72', 'POPNC', 'RB1_GE',
       'RP31DC', 'TGLS1', 'D1_D4FRS2', 'D1_D4TOS2', 'B11', 'P51N1111',
       'P51N1112', 'P51N11131', 'B5_GS1', 'P51N1113', 'GFSPR',
       'P51N1113O', 'PPPGDP', 'D1_D4NFRS2', 'D1S1', 'PCGNPA', 'B1_GE',
       'B1_GI', 'GDIS1', 'B1_GS1', 'P3', 'P3_P5', 'P311B', 'P312B',
       'P312N', 'P313B', 'P314B', 'P31DC', 'P31NC', 'P31S14',
       'P31S14_S15', 'P31S14_S15B', 'P31S15', 'P31S15B', 'P3S13', 'P5',
       'P51', 'P51A', 'P51S', 'P52', 'P52_P53', 'P5S1', 'RB1_GA',
       'RB1_GI', 'RB1_GS1', 'D5_D7FRS2', 'D5_D7TOS2', 'P51N1114',
       'P51N112', 'B1GVA', 'B1GVB_E', 'B1GVC', 'B1GVF', 'B1GVG_I',
       'B1GVG_U', 'B1GVJ', 'B1GVK', 'B1GVL', 'B1GVM_N', 'B1GVO_Q',
       'B1GVR_U', 'D11VA', 'D11VB_E', 'D11VC', 'D11VF', 'D11VG_I',
       'D11VJ', 'D11VK', 'D11VL', 'D11VM_N', 'D11VO_Q', 'D11VR_U', 'D1VA',
       'D1VB_E', 'D1VC', 'D1VF', 'D1VG_I', 'D1VJ', 'D1VK', 'D1VL',
       'D1VM_N', 'D1VO_Q', 'D1VR_U', 'EEMVA